- Sentence Extraction

In [ ]:
import pandas as pd
import Example
import sqlite3
# 신조어 목록 불러오기
df = pd.read_excel('new_words_var.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

# 전체 데이터 합치기
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
head = df1.append(df2)

# 불용어처리 없는 sentence 추출
sent = Example.extract_sent(head, new_words)
sent = pd.DataFrame.from_dict(sent, orient='index')
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)

- Generating Model using subtokenizer

In [ ]:
import sqlite3
import pandas as pd
from collections import defaultdict
import Example
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('sent.db')
temp = pd.read_sql('SELECT * FROM sent', conn)

sent = list()
for _ in temp['0']:
    sent.extend(_.split('  '))

In [ ]:
sgen = Example.generator_subtokenizer(sent)
X, y = sgen.create_var(sent)
sgen.model()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=13)
hist = sgen.model.fit(X_train, y_train, epochs=100, batch_size=128, validation_date=(X_test,y_test), verbose=2)

In [ ]:
Example.plot_history(['sgen':hist])

In [ ]:
sgen.sentence_generation('가성비', 10)

- Generating Model using ltokenizer

In [ ]:
import sqlite3
import pandas as pd
from collections import defaultdict
import Example
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('sent.db')
temp = pd.read_sql('SELECT * FROM sent', conn)

full_sentence = list()
example_sentence = list()
for _ in range(len(temp['0'])):
    sent = temp['0'][_]
    sent = sent.split('  ')
    sent = [_+'/' for _ in sent]
    words = {temp['index'][_]: 1.0}
    
    lgen = Example.generator_ltokenizer(words, sent)
    sentences = [[word for word in lgen.tokenizer(sentence)] for sentence in sent]
    lgen.train_word_model(sentences)
    X,y = lgen.create_var(sentences)
    lgen.model()
    lgen.model.fit(X, y, epochs=70, verbose=2)
    
    temp_sent = lgen.sentence_generation(temp['index'][_], 22)
    result = ''
    for _ in temp_sent:
        if _ == '/':
            break
        else:
            result = result + '' + _
    full_sentence.append(temp_sent)
    example_sentence.append(result)

training was done. used memory 0.578 Gbry 0.565 Gb
all cohesion probabilities was computed. # words = 1678
Epoch 1/70
609/609 - 29s - loss: 7.6120 - accuracy: 0.1422
Epoch 2/70
609/609 - 27s - loss: 7.0041 - accuracy: 0.1500
Epoch 3/70
609/609 - 23s - loss: 6.6258 - accuracy: 0.1628
Epoch 4/70
609/609 - 23s - loss: 6.2991 - accuracy: 0.1742
Epoch 5/70
609/609 - 23s - loss: 5.9964 - accuracy: 0.1825
Epoch 6/70
609/609 - 23s - loss: 5.6712 - accuracy: 0.1935
Epoch 7/70
609/609 - 23s - loss: 5.3267 - accuracy: 0.2113
Epoch 8/70
609/609 - 23s - loss: 4.9506 - accuracy: 0.2397
Epoch 9/70
609/609 - 23s - loss: 4.5708 - accuracy: 0.2736
Epoch 10/70
609/609 - 24s - loss: 4.1819 - accuracy: 0.3169
Epoch 11/70
609/609 - 23s - loss: 3.7806 - accuracy: 0.3644
Epoch 12/70
609/609 - 23s - loss: 3.3879 - accuracy: 0.4172
Epoch 13/70
609/609 - 24s - loss: 3.0136 - accuracy: 0.4681
Epoch 14/70
609/609 - 23s - loss: 2.6610 - accuracy: 0.5219
Epoch 15/70
609/609 - 23s - loss: 2.3515 - accuracy: 0.5698
Ep

Epoch 65/70
369/369 - 15s - loss: 0.7234 - accuracy: 0.8447
Epoch 66/70
369/369 - 15s - loss: 0.7165 - accuracy: 0.8448
Epoch 67/70
369/369 - 15s - loss: 0.7089 - accuracy: 0.8474
Epoch 68/70
369/369 - 15s - loss: 0.7125 - accuracy: 0.8465
Epoch 69/70
369/369 - 14s - loss: 0.7075 - accuracy: 0.8459
Epoch 70/70
369/369 - 15s - loss: 0.6993 - accuracy: 0.8477
training was done. used memory 0.477 Gbry 0.476 Gb
all cohesion probabilities was computed. # words = 532
Epoch 1/70
179/179 - 9s - loss: 7.0950 - accuracy: 0.1432
Epoch 2/70
179/179 - 4s - loss: 6.5356 - accuracy: 0.1455
Epoch 3/70
179/179 - 4s - loss: 6.2045 - accuracy: 0.1516
Epoch 4/70
179/179 - 4s - loss: 5.8726 - accuracy: 0.1609
Epoch 5/70
179/179 - 4s - loss: 5.5715 - accuracy: 0.1703
Epoch 6/70
179/179 - 4s - loss: 5.3163 - accuracy: 0.1801
Epoch 7/70
179/179 - 4s - loss: 5.0578 - accuracy: 0.1922
Epoch 8/70
179/179 - 4s - loss: 4.8130 - accuracy: 0.1998
Epoch 9/70
179/179 - 4s - loss: 4.5897 - accuracy: 0.2159
Epoch 10/70


Epoch 60/70
663/663 - 22s - loss: 1.0269 - accuracy: 0.7766
Epoch 61/70
663/663 - 22s - loss: 1.0217 - accuracy: 0.7788
Epoch 62/70
663/663 - 22s - loss: 1.0287 - accuracy: 0.7760
Epoch 63/70
663/663 - 22s - loss: 1.0243 - accuracy: 0.7766
Epoch 64/70
663/663 - 22s - loss: 1.0236 - accuracy: 0.7790
Epoch 65/70
663/663 - 22s - loss: 1.0093 - accuracy: 0.7792
Epoch 66/70
663/663 - 22s - loss: 1.0130 - accuracy: 0.7781
Epoch 67/70
663/663 - 22s - loss: 1.0008 - accuracy: 0.7806
Epoch 68/70
663/663 - 22s - loss: 0.9963 - accuracy: 0.7838
Epoch 69/70
663/663 - 22s - loss: 1.0040 - accuracy: 0.7785
Epoch 70/70
663/663 - 22s - loss: 0.9972 - accuracy: 0.7797
training was done. used memory 0.499 Gbry 0.495 Gb
all cohesion probabilities was computed. # words = 2140
Epoch 1/70
678/678 - 38s - loss: 6.9765 - accuracy: 0.1930
Epoch 2/70
678/678 - 32s - loss: 6.1473 - accuracy: 0.2284
Epoch 3/70
678/678 - 32s - loss: 5.7254 - accuracy: 0.2509
Epoch 4/70
678/678 - 32s - loss: 5.3470 - accuracy: 0.27

234/234 - 5s - loss: 1.0158 - accuracy: 0.7747
Epoch 55/70
234/234 - 5s - loss: 1.0081 - accuracy: 0.7742
Epoch 56/70
234/234 - 5s - loss: 0.9973 - accuracy: 0.7790
Epoch 57/70
234/234 - 5s - loss: 0.9949 - accuracy: 0.7751
Epoch 58/70
234/234 - 5s - loss: 0.9882 - accuracy: 0.7824
Epoch 59/70
234/234 - 5s - loss: 0.9800 - accuracy: 0.7820
Epoch 60/70
234/234 - 5s - loss: 0.9637 - accuracy: 0.7842
Epoch 61/70
234/234 - 5s - loss: 0.9701 - accuracy: 0.7829
Epoch 62/70
234/234 - 5s - loss: 0.9621 - accuracy: 0.7837
Epoch 63/70
234/234 - 5s - loss: 0.9488 - accuracy: 0.7865
Epoch 64/70
234/234 - 5s - loss: 0.9450 - accuracy: 0.7834
Epoch 65/70
234/234 - 5s - loss: 0.9442 - accuracy: 0.7864
Epoch 66/70
234/234 - 5s - loss: 0.9375 - accuracy: 0.7869
Epoch 67/70
234/234 - 5s - loss: 0.9363 - accuracy: 0.7844
Epoch 68/70
234/234 - 5s - loss: 0.9228 - accuracy: 0.7905
Epoch 69/70
234/234 - 5s - loss: 0.9272 - accuracy: 0.7876
Epoch 70/70
234/234 - 5s - loss: 0.9186 - accuracy: 0.7850
training 

Epoch 50/70
327/327 - 8s - loss: 1.4699 - accuracy: 0.7204
Epoch 51/70
327/327 - 8s - loss: 1.4676 - accuracy: 0.7212
Epoch 52/70
327/327 - 8s - loss: 1.4507 - accuracy: 0.7242
Epoch 53/70
327/327 - 8s - loss: 1.4533 - accuracy: 0.7223
Epoch 54/70
327/327 - 8s - loss: 1.4484 - accuracy: 0.7237
Epoch 55/70
327/327 - 8s - loss: 1.4519 - accuracy: 0.7244
Epoch 56/70
327/327 - 8s - loss: 1.4418 - accuracy: 0.7238
Epoch 57/70
327/327 - 8s - loss: 1.4417 - accuracy: 0.7236
Epoch 58/70
327/327 - 8s - loss: 1.4322 - accuracy: 0.7243
Epoch 59/70
327/327 - 8s - loss: 1.4377 - accuracy: 0.7251
Epoch 60/70
327/327 - 8s - loss: 1.4337 - accuracy: 0.7234
Epoch 61/70
327/327 - 8s - loss: 1.4303 - accuracy: 0.7244
Epoch 62/70
327/327 - 8s - loss: 1.4325 - accuracy: 0.7233
Epoch 63/70
327/327 - 8s - loss: 1.4303 - accuracy: 0.7237
Epoch 64/70
327/327 - 8s - loss: 1.4179 - accuracy: 0.7236
Epoch 65/70
327/327 - 8s - loss: 1.4238 - accuracy: 0.7282
Epoch 66/70
327/327 - 8s - loss: 1.4218 - accuracy: 0.72

...

Epoch 60/70
2885/2885 - 245s - loss: 1.7145 - accuracy: 0.6915
Epoch 61/70
2885/2885 - 247s - loss: 1.7025 - accuracy: 0.6935
Epoch 62/70
2885/2885 - 246s - loss: 1.6987 - accuracy: 0.6946
Epoch 63/70
2885/2885 - 245s - loss: 1.6972 - accuracy: 0.6956
Epoch 64/70
2885/2885 - 247s - loss: 1.6925 - accuracy: 0.6937
Epoch 65/70
2885/2885 - 246s - loss: 1.6871 - accuracy: 0.6961
Epoch 66/70
2885/2885 - 245s - loss: 1.6757 - accuracy: 0.6989
Epoch 67/70
2885/2885 - 246s - loss: 1.6775 - accuracy: 0.6982
Epoch 68/70
2885/2885 - 248s - loss: 1.6687 - accuracy: 0.7000
Epoch 69/70
2885/2885 - 246s - loss: 1.6631 - accuracy: 0.7010
Epoch 70/70
2885/2885 - 246s - loss: 1.6566 - accuracy: 0.7028
training was done. used memory 0.606 Gbry 0.604 Gb
all cohesion probabilities was computed. # words = 1594
Epoch 1/70
567/567 - 24s - loss: 7.5154 - accuracy: 0.1357
Epoch 2/70
567/567 - 18s - loss: 6.8332 - accuracy: 0.1445
Epoch 3/70
567/567 - 18s - loss: 6.4093 - accuracy: 0.1615
Epoch 4/70
567/567 - 18

Epoch 51/70
4593/4593 - 472s - loss: 2.1895 - accuracy: 0.5920
Epoch 52/70
4593/4593 - 474s - loss: 2.1863 - accuracy: 0.5936
Epoch 53/70
4593/4593 - 483s - loss: 2.1834 - accuracy: 0.5937
Epoch 54/70
4593/4593 - 474s - loss: 2.1793 - accuracy: 0.5944
Epoch 55/70
4593/4593 - 473s - loss: 2.1719 - accuracy: 0.5960
Epoch 56/70
4593/4593 - 474s - loss: 2.1688 - accuracy: 0.5962
Epoch 57/70
4593/4593 - 474s - loss: 2.1661 - accuracy: 0.5972
Epoch 58/70
4593/4593 - 472s - loss: 2.1626 - accuracy: 0.5979
Epoch 59/70
4593/4593 - 475s - loss: 2.1558 - accuracy: 0.5971
Epoch 60/70
4593/4593 - 478s - loss: 2.1580 - accuracy: 0.5967
Epoch 61/70
4593/4593 - 474s - loss: 2.1527 - accuracy: 0.5989
Epoch 62/70
4593/4593 - 472s - loss: 2.1451 - accuracy: 0.6011
Epoch 63/70
4593/4593 - 479s - loss: 2.1451 - accuracy: 0.6002
Epoch 64/70
4593/4593 - 473s - loss: 2.1459 - accuracy: 0.6001
Epoch 65/70
4593/4593 - 474s - loss: 2.1437 - accuracy: 0.6002
Epoch 66/70
4593/4593 - 473s - loss: 2.1444 - accuracy:

Epoch 43/70
3034/3034 - 232s - loss: 1.8864 - accuracy: 0.6333
Epoch 44/70
3034/3034 - 234s - loss: 1.8725 - accuracy: 0.6339
Epoch 45/70
3034/3034 - 233s - loss: 1.8502 - accuracy: 0.6393
Epoch 46/70
3034/3034 - 233s - loss: 1.8351 - accuracy: 0.6402
Epoch 47/70
3034/3034 - 232s - loss: 1.8346 - accuracy: 0.6414
Epoch 48/70
3034/3034 - 234s - loss: 1.8119 - accuracy: 0.6437
Epoch 49/70
3034/3034 - 235s - loss: 1.7957 - accuracy: 0.6468
Epoch 50/70
3034/3034 - 236s - loss: 1.7924 - accuracy: 0.6483
Epoch 51/70
3034/3034 - 232s - loss: 1.7725 - accuracy: 0.6523
Epoch 52/70
3034/3034 - 234s - loss: 1.7666 - accuracy: 0.6544
Epoch 53/70
3034/3034 - 234s - loss: 1.7576 - accuracy: 0.6550
Epoch 54/70
3034/3034 - 232s - loss: 1.7448 - accuracy: 0.6563
Epoch 55/70
3034/3034 - 232s - loss: 1.7416 - accuracy: 0.6577
Epoch 56/70
3034/3034 - 233s - loss: 1.7318 - accuracy: 0.6591
Epoch 57/70
3034/3034 - 232s - loss: 1.7199 - accuracy: 0.6621
Epoch 58/70
3034/3034 - 232s - loss: 1.7136 - accuracy:

Epoch 37/70
135/135 - 2s - loss: 1.2806 - accuracy: 0.7290
Epoch 38/70
135/135 - 2s - loss: 1.2438 - accuracy: 0.7369
Epoch 39/70
135/135 - 2s - loss: 1.2091 - accuracy: 0.7339
Epoch 40/70
135/135 - 2s - loss: 1.1758 - accuracy: 0.7446
Epoch 41/70
135/135 - 2s - loss: 1.1492 - accuracy: 0.7411
Epoch 42/70
135/135 - 2s - loss: 1.1273 - accuracy: 0.7495
Epoch 43/70
135/135 - 2s - loss: 1.1179 - accuracy: 0.7464
Epoch 44/70
135/135 - 2s - loss: 1.0884 - accuracy: 0.7625
Epoch 45/70
135/135 - 2s - loss: 1.0586 - accuracy: 0.7611
Epoch 46/70
135/135 - 2s - loss: 1.0363 - accuracy: 0.7683
Epoch 47/70
135/135 - 2s - loss: 1.0362 - accuracy: 0.7676
Epoch 48/70
135/135 - 2s - loss: 1.0280 - accuracy: 0.7600
Epoch 49/70
135/135 - 2s - loss: 0.9977 - accuracy: 0.7711
Epoch 50/70
135/135 - 2s - loss: 0.9836 - accuracy: 0.7714
Epoch 51/70
135/135 - 2s - loss: 0.9707 - accuracy: 0.7769
Epoch 52/70
135/135 - 2s - loss: 0.9370 - accuracy: 0.7856
Epoch 53/70
135/135 - 2s - loss: 0.9559 - accuracy: 0.78

Epoch 31/70
3965/3965 - 405s - loss: 2.4510 - accuracy: 0.5617
Epoch 32/70
3965/3965 - 465s - loss: 2.4110 - accuracy: 0.5686
Epoch 33/70
3965/3965 - 434s - loss: 2.3887 - accuracy: 0.5702
Epoch 34/70


In [ ]:
lgen = Example.generator_ltokenizer(words, sent)
sentences = [[word for word in lgen.tokenizer(sentence)] for sentence in sent]
lgen.train_word_model(sentences)
X,y = lgen.create_var(sentences)
lgen.model()
lgen.model.fit(X, y, epochs=70, verbose=2)

In [ ]:
temp = lgen.sentence_generation('가스라이팅', 22)
result = ''
for _ in temp:
    if _ == '/':
        break
    else:
        result = result + '' + _
full_sentence.append(temp)
example_sentence.append(result)